In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

**bold text**
# Language Modeling with ``nn.Transformer`` and torchtext

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

This is a tutorial on training a sequence-to-sequence model that uses the
[nn.Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)_ module.

The PyTorch 1.2 release includes a standard transformer module based on the
paper [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf)_.
Compared to Recurrent Neural Networks (RNNs), the transformer model has proven
to be superior in quality for many sequence-to-sequence tasks while being more
parallelizable. The ``nn.Transformer`` module relies entirely on an attention
mechanism (implemented as
[nn.MultiheadAttention](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)_)
to draw global dependencies between input and output. The ``nn.Transformer``
module is highly modularized such that a single component (e.g.,
[nn.TransformerEncoder](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoder.html)_)
can be easily adapted/composed.

<img src="file://../_static/img/transformer_architecture.jpg">


## Define the model




In [2]:
%%capture installs
%pip install polars
%pip install torchviz
# %conda install -y conda install -c  python-graphviz
# %conda install -c fastchan python-graphviz -y

In this tutorial, we train a ``nn.TransformerEncoder`` model on a
language modeling task. The language modeling task is to assign a
probability for the likelihood of a given word (or a sequence of words)
to follow a sequence of words. A sequence of tokens are passed to the embedding
layer first, followed by a positional encoding layer to account for the order
of the word (see the next paragraph for more details). The
``nn.TransformerEncoder`` consists of multiple layers of
[nn.TransformerEncoderLayer](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html)_.
Along with the input sequence, a square attention mask is required because the
self-attention layers in ``nn.TransformerEncoder`` are only allowed to attend
the earlier positions in the sequence. For the language modeling task, any
tokens on the future positions should be masked. To produce a probability
distribution over output words, the output of the ``nn.TransformerEncoder``
model is passed through a linear layer followed by a log-softmax function.




In [3]:
import math
import os
import time
from tempfile import TemporaryDirectory
from typing import Tuple

import re
from numbers import Number

from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

from tqdm import trange, tqdm

import torch
import polars as pl
import numpy as np

from torch.utils.data import DataLoader, Dataset

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset



In [4]:
torch.__version__

'2.0.0'

In [5]:
if torch.backends.mps.is_built():
    device_name = 'mps'
elif torch.cuda.is_available():
    device_name = 'cuda'
else:
    device_name = 'cpu'
device = torch.device(device_name)
print(device)

cuda


In [6]:
weather = pl.read_parquet("~/Hephaestus/data/weather_clean.parquet")
weather.head()


x,y,station_name,climate_identifier,province_code,local_year,local_month,local_day,local_hour,temp,temp_flag,dew_point_temp,dew_point_temp_flag,humidex,precip_amount,precip_amount_flag,relative_humidity,relative_humidity_flag,station_pressure,station_pressure_flag,wind_chill,wind_direction,wind_direction_flag,wind_speed,wind_speed_flag
f64,f64,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,str,f64,str,f64,str,f64,f64,str,f64,str
-114.000297,51.109447,"""CALGARY INT'L …","""3031094""","""AB""","""2010""","""1""","""1""","""0""",-21.6,"""missing""",-23.9,"""missing""",null,null,"""missing""",82.0,"""missing""",89.38,"""missing""",null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L …","""3031094""","""AB""","""2010""","""1""","""1""","""1""",-21.2,"""missing""",-23.5,"""missing""",null,null,"""missing""",82.0,"""missing""",89.25,"""missing""",null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L …","""3031094""","""AB""","""2010""","""1""","""1""","""2""",-20.8,"""missing""",-23.0,"""missing""",null,null,"""missing""",82.0,"""missing""",89.21,"""missing""",null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L …","""3031094""","""AB""","""2010""","""1""","""1""","""3""",-20.4,"""missing""",-22.6,"""missing""",null,null,"""missing""",83.0,"""missing""",89.12,"""missing""",null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L …","""3031094""","""AB""","""2010""","""1""","""1""","""4""",-20.4,"""missing""",-22.7,"""missing""",null,null,"""missing""",82.0,"""missing""",89.04,"""missing""",null,null,"""M""",null,"""M"""


``PositionalEncoding`` module injects some information about the
relative or absolute position of the tokens in the sequence. The
positional encodings have the same dimension as the embeddings so that
the two can be summed. Here, we use ``sine`` and ``cosine`` functions of
different frequencies.




In [7]:
def scale_numeric(df):
    for col in df.columns:
        if df[col].dtype == pl.Float64 or df[col].dtype == pl.Int64:
            df = df.with_columns(
                ((pl.col(col) - pl.col(col).mean()) / pl.col(col).std()).alias(col)
            )  # .select(pl.col(["dew_point_temp", "NewCOL"]))
    return df


weather = scale_numeric(weather)


In [8]:
def make_lower_remove_special_chars(df):
    df = df.with_columns(
        pl.col(pl.Utf8).str.to_lowercase().str.replace_all("[^a-zA-Z0-9]", " ")
    )
    return df


weather = make_lower_remove_special_chars(weather)

In [9]:
def get_unique_utf8_values(df):
    arr = np.array([])
    for col in df.select(pl.col(pl.Utf8)).columns:
        arr = np.append(arr, df[col].unique().to_numpy())

    return np.unique(arr)


weather_val_tokens = get_unique_utf8_values(weather)

In [10]:
def get_col_tokens(df):
    tokens = []
    for col_name in df.columns:
        sub_strs = re.split(r"[^a-zA-Z0-9]", col_name)
        tokens.extend(sub_strs)
    return np.unique(np.array(tokens))


weather_col_tokens = get_col_tokens(weather)

In [11]:
special_tokens = np.array(
    [
        "missing",
        "<mask>"
        "<pad>",
        "<unk>",
        ":",
        ",",
        "<row-start>",
        "<row-end>",
    ]
)
tokens = np.unique(
    np.concatenate(
        (
            weather_val_tokens,
            weather_col_tokens,
            special_tokens,
        )
    )
)
tokens


array([',', '0', '1', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '2', '20', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30',
       '3012206', '3026knq', '3031094', '3033890', '3035208', '3062696',
       '31', '4', '5', '6', '7', '8', '9', ':', '<mask><pad>',
       '<row-end>', '<row-start>', '<unk>', 'ab', 'amount',
       'calgary int l cs', 'chill', 'climate', 'code', 'day', 'dew',
       'direction', 'edmonton international cs', 'flag',
       'fort mcmurray cs', 'hour', 'humidex', 'humidity', 'identifier',
       'lethbridge cda', 'local', 'm', 'missing', 'month', 'name',
       'pincher creek climate', 'point', 'precip', 'pressure', 'province',
       'relative', 'speed', 'station', 'sundre a', 'temp', 'wind', 'x',
       'y', 'year'], dtype=object)

In [12]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 100_000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [13]:

# class TransformerModel(nn.Module):

#     def __init__(self, n_token: int, d_model: int, n_head: int, d_hid: int,
#                 n_layers: int, dropout: float = 0.15):
#         super().__init__()
#         self.n_token = n_token +1
#         self.model_type = 'Transformer'
#         self.pos_encoder = PositionalEncoding(d_model, dropout)
#         encoder_layers = TransformerEncoderLayer(d_model, n_head, d_hid, dropout)
#         self.transformer_encoder = TransformerEncoder(encoder_layers, n_layers)
#         self.encoder = StringNumericEmbedding(n_token, d_model)
#         self.d_model = d_model
#         self.decoder = nn.Linear(d_model, n_token)
#         self.numeric_decoder = nn.Linear(d_model, n_token)

#         # self.numeric_decoder = nn.Linear(d_model)


#         self.init_weights()

#     def init_weights(self) -> None:
#         initrange = 0.1
#         self.encoder.embedding.weight.data.uniform_(-initrange, initrange)
#         self.decoder.bias.data.zero_()
#         self.decoder.weight.data.uniform_(-initrange, initrange)
#         # self.numeric_decoder.data.uniform(-initrange, initrange)

#     def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
#         """
#         Arguments:
#             src: Tensor, shape ``[seq_len, batch_size]``
#             src_mask: Tensor, shape ``[seq_len, seq_len]``

#         Returns:
#             output Tensor of shape ``[seq_len, batch_size, ntoken]``
#         """
#         # src_shape = src.shape
#         print(f"raw src_shape: {len(src)}")
#         src = self.encoder(src) * math.sqrt(self.d_model)
#         print(f"encoded src_shape: {src.shape}")

#         src_shape = src.shape
#         src = self.pos_encoder(src)
#         output = self.transformer_encoder(src)
#         print(f"output_shape: {output.shape}")
#         numeric_output = self.numeric_decoder(output)  # .flatten()
#         numeric_output = torch.mean(numeric_output, [1, 2])
#         # numeric_output = nn.flatten(numeric_output)
#         output = self.decoder(output)
#         print(f"output_shape decoded: {output.shape}")
#         output = output.view(-1, self.n_token+1)
#         # output = output.view(-1, src_shape[0]).T
#         print(f"output_shape view: {output.shape}")

#         return output, numeric_output


# def generate_square_subsequent_mask(sz: int) -> Tensor:
#     """Generates an upper-triangular matrix of ``-inf``, with zeros on ``diag``."""
#     return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [14]:
@dataclass
class StringNumeric:
    value: Union[str, float]
    # all_tokens: np.array
    is_numeric: bool = field(default=None, repr=True)
    embedding_idx: int = field(default=None, repr=True)
    is_special: bool = field(default=False, repr=True)
    def __post_init__(self):
        if isinstance(self.value, str):
            self.is_numeric = False
        else:
            self.is_numeric = True
            self.embedding_idx = 0

    def gen_embed_idx(self, tokens: np.array, special_tokens: np.array):
        if not self.is_numeric:
            try:
                self.embedding_idx = np.where(tokens == self.value)[0][0] + 1
            except IndexError:
                self.embedding_idx = np.where(tokens == "<unk>")[0][0] + 1
            if self.value in special_tokens:
                self.is_special = True



x = StringNumeric(value="climate")
# xx = StringNumeric(value="climate", tokens=tokens)
print(x)
y = StringNumeric(value=1.0)
print(y)
z = StringNumeric(value="SomeRandomString")
print(z)
x.gen_embed_idx(tokens, special_tokens)
print(x)
# print(StringNumeric(value=1.0, all_tokens=tokens))


StringNumeric(value='climate', is_numeric=False, embedding_idx=None, is_special=False)
StringNumeric(value=1.0, is_numeric=True, embedding_idx=0, is_special=False)
StringNumeric(value='SomeRandomString', is_numeric=False, embedding_idx=None, is_special=False)
StringNumeric(value='climate', is_numeric=False, embedding_idx=62, is_special=False)


In [15]:
class TabularDataset(Dataset):
    # def __init__(self, df: pl.DataFrame, vocab_dict: Dict, m_dim: int) -> Dataset:
    def __init__(
        self,
        df: pl.DataFrame,
        vocab,
        special_tokens: np.array,
        shuffle_cols=False,
        max_row_length=512,
    ) -> Dataset:
        self.df = df
        self.vocab = vocab
        self.special_tokens = special_tokens
        self.vocab_len = vocab.shape[0]
        self.shuffle_cols = shuffle_cols
        self.max_row_length = max_row_length
        # self.vocab_dict = vocab_dict
        # self.embedding = nn.Embedding(len(self.string_vocab), m_dim)
        # Numeric Scale

        # self.col_vocab = self.df.columns

    def __len__(self):
        """Returns the number of sequences in the dataset."""
        length = self.df.shape[0]
        return length

    def __getitem__(self, idx):
        """Returns a tuple of (input, target) at the given index."""
        row = self.df[idx]
        row = self.splitter(row)
        return row



    def splitter(self, row: pl.DataFrame) -> List[Union[str, float, None]]:
        vals = ["<row-start>"]
        cols = row.columns
        if self.shuffle_cols:
            np.random.shuffle(cols)

        for col in cols:
            value = row[col][0]
            col = col.split("_")
            vals.extend(col)
            vals.append(":")
            if isinstance(value, Number):
                vals.append(value)
            elif value is None:
                vals.append("missing")
                # Nones are only for numeric columns, others are "None"
            elif isinstance(value, str):
                vals.extend(value.split(" "))
            else:
                raise ValueError("Unknown type")
            vals.append(",")
        vals.append("<row-end>")

        val_len = len(vals)
        if val_len < self.max_row_length:
            diff = self.max_row_length - val_len
            vals.extend(["<pad>"] * diff)
        elif val_len > self.max_row_length:
            vals = vals[:self.max_row_length - 1]
            # add warning

            vals = np.append(vals, ["<row-end>"])
            print("Row too long, truncating")
            Warning("Row too long, truncating")
        vals = [StringNumeric(value=val) for val in vals]
        for val in vals:
            val.gen_embed_idx(self.vocab, self.special_tokens)

        return vals


weather_ds = TabularDataset(weather, tokens, special_tokens=special_tokens, shuffle_cols=False, max_row_length=140 )

print(len(weather_ds[0]))

140


In [16]:
weather_ds.vocab

array([',', '0', '1', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '2', '20', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30',
       '3012206', '3026knq', '3031094', '3033890', '3035208', '3062696',
       '31', '4', '5', '6', '7', '8', '9', ':', '<mask><pad>',
       '<row-end>', '<row-start>', '<unk>', 'ab', 'amount',
       'calgary int l cs', 'chill', 'climate', 'code', 'day', 'dew',
       'direction', 'edmonton international cs', 'flag',
       'fort mcmurray cs', 'hour', 'humidex', 'humidity', 'identifier',
       'lethbridge cda', 'local', 'm', 'missing', 'month', 'name',
       'pincher creek climate', 'point', 'precip', 'pressure', 'province',
       'relative', 'speed', 'station', 'sundre a', 'temp', 'wind', 'x',
       'y', 'year'], dtype=object)

In [17]:
class StringNumericEmbedding(nn.Module):
    def __init__(self, n_token: int, d_model: int):
        super().__init__()
        self.embedding = nn.Embedding(n_token+1, d_model, padding_idx=0).to(device)

    def forward(self, input: StringNumeric):
        embedding_index = torch.tensor([i.embedding_idx for i in input]).to(device)
        embed = self.embedding(embedding_index)
        with torch.no_grad():
            for idx, value in enumerate(input):
                if value.is_numeric:
                    embed[idx][0] = value.value
        return embed

my_embed = StringNumericEmbedding(weather_ds.vocab_len+1, 32)

t = my_embed(weather_ds[0])


In [18]:
t.shape

torch.Size([140, 32])

In [19]:
def mask_row(row):
    prob = 0.15
    for idx, val in enumerate(row):
        if val.is_special:
            continue
        if np.random.rand() < prob:
            val = StringNumeric(value="<mask>")
            val.gen_embed_idx(tokens, special_tokens)
            row[idx] = val
    return row

x = mask_row(weather_ds[0])
x[:3]

[StringNumeric(value='<row-start>', is_numeric=False, embedding_idx=56, is_special=True),
 StringNumeric(value='x', is_numeric=False, embedding_idx=91, is_special=False),
 StringNumeric(value=':', is_numeric=False, embedding_idx=53, is_special=True)]

In [20]:
for idx, val in enumerate(x):
    if val == weather_ds[0][idx]:
        continue
    else:
        print(f"{val} != {weather_ds[0][idx]}")

StringNumeric(value='<mask>', is_numeric=False, embedding_idx=57, is_special=False) != StringNumeric(value='int', is_numeric=False, embedding_idx=57, is_special=False)
StringNumeric(value='<mask>', is_numeric=False, embedding_idx=57, is_special=False) != StringNumeric(value='l', is_numeric=False, embedding_idx=57, is_special=False)
StringNumeric(value='<mask>', is_numeric=False, embedding_idx=57, is_special=False) != StringNumeric(value='year', is_numeric=False, embedding_idx=93, is_special=False)
StringNumeric(value='<mask>', is_numeric=False, embedding_idx=57, is_special=False) != StringNumeric(value='local', is_numeric=False, embedding_idx=75, is_special=False)
StringNumeric(value='<mask>', is_numeric=False, embedding_idx=57, is_special=False) != StringNumeric(value='day', is_numeric=False, embedding_idx=64, is_special=False)
StringNumeric(value='<mask>', is_numeric=False, embedding_idx=57, is_special=False) != StringNumeric(value='dew', is_numeric=False, embedding_idx=65, is_specia

## Load and batch data




In [21]:
def batch_data(ds, idx: int, n_row=4):
    target = []
    for i in range(idx, idx+n_row):
        target.extend(ds[i])
    batch = mask_row(target)

    return batch, target


The model hyperparameters are defined below. The ``vocab`` size is
equal to the length of the vocab object.




In [22]:

data, targets = batch_data(weather_ds, idx=0, n_row=50)
print(len(data), len(targets))
# data[0]

7000 7000


In [23]:
data[0]

StringNumeric(value='<row-start>', is_numeric=False, embedding_idx=56, is_special=True)

In [24]:
my_embed = StringNumericEmbedding(weather_ds.vocab_len+1, 32)

t = my_embed(data)

In [25]:
t.shape

torch.Size([7000, 32])

In [77]:

class TransformerModel(nn.Module):

    def __init__(self, n_token: int, d_model: int, n_head: int, d_hid: int,
                n_layers: int, dropout: float = 0.15):
        super().__init__()
        n_token = n_token +1
        self.n_token = n_token
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, n_head, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, n_layers)
        self.encoder = StringNumericEmbedding(n_token, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, n_token)
        self.numeric_decoder = nn.Linear(d_model, n_token)
        self.numeric_flattener = nn.Linear(n_token, 1)

        # self.numeric_decoder = nn.Linear(d_model)


        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        # self.numeric_decoder.data.uniform(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """
        # src_shape = src.shape
        # print(f"raw src_shape: {len(src)}")
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = torch.unsqueeze(src, dim=1)
        # print(f"encoded src_shape: {src.shape}")

        src_shape = src.shape
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        # print(f"output_shape: {output.shape}")
        numeric_output = self.numeric_decoder(output)  # .flatten()
        numeric_output = torch.squeeze(numeric_output, dim=1)
        # numeric_output = torch.mean(numeric_output, [1])
        numeric_output = self.numeric_flattener(numeric_output)
        # numeric_output = nn.flatten(numeric_output)
        output = self.decoder(output)
        output = torch.squeeze(output, dim=1)
        numeric_output = numeric_output.view(output.shape[0])

        # print(f"output_shape decoded: {output.shape}")
        # output = output.view(-1, self.n_token+1)
        # output = output.view(-1, src_shape[0]).T
        # print(f"output_shape view: {output.shape}")

        return output, numeric_output


In [78]:
cat.shape

torch.Size([140, 94])

In [79]:
n_token = len(weather_ds.vocab)  # size of vocabulary
d_model = 32  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
n_layers = 4  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
n_head = 4  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
model = TransformerModel(n_token, d_model, n_head, d_hid, n_layers, dropout).to(device)
initial_memory_allocated = torch.cuda.memory_allocated()
initial_memory_reserved = torch.cuda.memory_reserved()
c,n = model(data)
final_memory_allocated = torch.cuda.memory_allocated()
final_memory_reserved = torch.cuda.memory_reserved()
print(f'Increase in memory allocated: {final_memory_allocated - initial_memory_allocated}')
print(f'Increase in memory reserved: {final_memory_reserved - initial_memory_reserved}')

Increase in memory allocated: 4736512
Increase in memory reserved: 0


In [80]:
c.shape, n.shape

(torch.Size([140, 94]), torch.Size([140]))

## Run the model




In [81]:
# from torchviz import make_dot

# make_dot((c,n), params=dict(list(model.named_parameters())))# .render("rnn_torchviz", format="png")

We use [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)_
with the [SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)_
(stochastic gradient descent) optimizer. The learning rate is initially set to
5.0 and follows a [StepLR](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html)_
schedule. During training, we use [nn.utils.clip_grad_norm\_](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)_
to prevent gradients from exploding.




In [85]:
def custom_loss(class_preds, numeric_preds, raw_data):
    cross_entropy = nn.CrossEntropyLoss()
    mse_loss = nn.MSELoss()
    
    class_target = torch.tensor([i.embedding_idx for i in raw_data]).to(device)
    class_loss = cross_entropy(class_preds, class_target)
    
    actual_num_idx = torch.tensor([idx for idx, j in enumerate(raw_data) if j.is_numeric]).to(device)
    pred_nums = numeric_preds[actual_num_idx]
    # print(actual_num_idx.shape)
    actual_nums = torch.tensor([i.value for i in raw_data if i.is_numeric]).to(device)
    # print(actual_nums.shape)
    # print(pred_nums.shape)
    reg_loss = mse_loss(pred_nums, actual_nums)
    reg_loss_adjuster = 1 # class_loss/reg_loss
    
    return reg_loss*reg_loss_adjuster+class_loss, {"reg_loss": reg_loss,
                                                  "class_loss": class_loss}

# loss, d = custom_loss(c, n, data)
# custom_loss(c, n, data)


In [86]:
import copy
import time


lr = 0.99  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size =100, gamma=0.5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.99, patience=10, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0.01, eps=1e-08, verbose=False)
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 50
    n_row=50
    start_time = time.time()
    # src_mask = generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(trange(0, len(weather_ds) - 1, n_row)):
        
    # for batch, i in enumerate(range(len(weather_ds))):
        
        data, targets = batch_data(weather_ds, i, n_row=n_row)
        class_output, numeric_output = model(data)
        loss, loss_dict = custom_loss(class_output, numeric_output, targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            # lr = scheduler.get_last_lr()[0]
            lr = optimizer.param_groups[0]["lr"]
            
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(#f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | ',
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}', loss_dict)
            total_loss = 0
            start_time = time.time()
            scheduler.step(loss)



Loop over epochs. Save the model if the validation loss is the best
we've seen so far. Adjust the learning rate after each epoch.



In [ ]:
train(model)

  0%|          | 51/12714 [00:21<1:27:50,  2.40it/s]

lr 0.99 | ms/batch 432.40 |  loss  1.73 | ppl     5.65 {'reg_loss': tensor(0.2929, device='cuda:0', grad_fn=<MseLossBackward0>), 'class_loss': tensor(0.0171, device='cuda:0', grad_fn=<NllLossBackward0>)}


  1%|          | 101/12714 [00:43<1:35:40,  2.20it/s]

lr 0.99 | ms/batch 430.10 |  loss  0.89 | ppl     2.42 {'reg_loss': tensor(1.4536, device='cuda:0', grad_fn=<MseLossBackward0>), 'class_loss': tensor(0.0149, device='cuda:0', grad_fn=<NllLossBackward0>)}


  1%|          | 151/12714 [01:04<1:28:53,  2.36it/s]

...

  9%|▉         | 1201/12714 [08:37<1:21:14,  2.36it/s]

lr 0.99 | ms/batch 430.70 |  loss  0.35 | ppl     1.42 {'reg_loss': tensor(0.1703, device='cuda:0', grad_fn=<MseLossBackward0>), 'class_loss': tensor(0.0440, device='cuda:0', grad_fn=<NllLossBackward0>)}


 10%|▉         | 1251/12714 [08:59<1:20:06,  2.38it/s]

lr 0.99 | ms/batch 430.89 |  loss  0.33 | ppl     1.39 {'reg_loss': tensor(0.2437, device='cuda:0', grad_fn=<MseLossBackward0>), 'class_loss': tensor(0.0190, device='cuda:0', grad_fn=<NllLossBackward0>)}


 10%|▉         | 1256/12714 [09:01<1:20:09,  2.38it/s]

In [36]:
data, targets = batch_data(weather_ds, idx=0, n_row=1)
print(len(data), len(targets))
# data[0]

140 140


In [37]:
cat, num = model(data)


In [39]:
cat.shape, num.shape

(torch.Size([140, 94]), torch.Size([140]))

In [40]:
custom_loss(cat, num, targets)

(tensor(0.0077, device='cuda:0', grad_fn=<AddBackward0>),
 {'reg_loss': tensor(2.0211, device='cuda:0', grad_fn=<MseLossBackward0>),
  'class_loss': tensor(0.0038, device='cuda:0', grad_fn=<NllLossBackward0>)})

In [57]:
lsm = nn.LogSoftmax(dim=0)

In [58]:
lsm(cat).shape

torch.Size([140, 94])

In [ ]:
def evaluate(model, data, i):
    model.eval
    with torch.no_grad():
        data, targets = batch_data(weather_ds, i, n_row=4)
        class_output, numeric_output = model(data)
        loss, loss_dict = custom_loss(class_output, numeric_output, targets)
        
        return data, targets, class_output, numeric_output, loss, loss_dict
data, targets, class_output, numeric_output, loss, loss_dict = evaluate(model, weather_ds, 1)
        

In [ ]:
torch.softmax(class_output, 1).shape

In [67]:
m = nn.LogSoftmax(dim=1)
input = torch.randn(2, 3)
test_out = m(input)

In [70]:
test_out.sum(dim=1)

tensor([-4.0576, -4.8689])

In [71]:
input, test_out

(tensor([[-0.9935,  0.5413,  0.8352],
         [-0.5195, -1.0162,  1.3274]]),
 tensor([[-2.4737, -0.9389, -0.6450],
         [-2.0730, -2.5697, -0.2261]]))

In [ ]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, n_token)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

## Evaluate the best model on the test dataset




In [ ]:
test_loss = evaluate(model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

In [ ]:
class StringNumericEmbedding(nn.Embedding):
    def __init__(self, string_numeric:StringNumeric, embed_dim: int=28):
